In [1]:
from unsloth import FastLanguageModel # API для подгрузки модели

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-15 11:22:03 __init__.py:190] Automatically detected platform cuda.


In [2]:
max_seq_length = 4096
max_summary_length = 256
dtype = None
load_in_4bit = True

model_sft, tokenizer_sft = FastLanguageModel.from_pretrained(
    model_name = "qwen2_5_lora_sft_model",
    attn_implementation = "flash_attention_2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

model_sft = FastLanguageModel.for_inference(model_sft)

==((====))==  Unsloth 2025.2.9: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.2.9 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [3]:
alpaca_prompt = """Ниже приведена инструкция, описывающая задание, в сочетании с вводными данными, которые обеспечивают дальнейший контекст. Напишите ответ, который соответствующим образом выполнит запрос.

### Инструкция:
{}

### Вводные данные:
{}

### Ответ:
{}"""

In [4]:
def inference(
        input_prompt: str,
        prefix: str,
        max_new_tokens: int,
        temperature: float,
        top_p: float,
        top_k: int,
        repetition_penalty: float,
        skip_special: bool,
        num_beams: int,
        do_sample: bool):
    
    inputs = tokenizer_sft(
    [
        alpaca_prompt.format(
            prefix,
            input_prompt,
            "",
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model_sft.generate(
        input_ids = inputs.input_ids, 
        attention_mask = inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        use_cache=True)
    return tokenizer_sft.batch_decode(outputs, skip_special_tokens=skip_special)[0].split("### Ответ:\n")[-1]

In [ ]:
from app import lanch_language_model_server

lanch_language_model_server(inference, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://384603c2c7f4684d74.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


: 